In [3]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from transformers import ViTForImageClassification, ViTImageProcessor
from tqdm import tqdm
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_curve, auc
)

# Configuration
CONFIG = {
    'seed': 42,
    'model_name': "google/vit-base-patch16-224",
    'batch_size': 16,
    'num_epochs': 100,
    'learning_rate': 1e-5,  # Lowered learning rate for fine-tuning
    'weight_decay': 1e-2,
    'train_dir': '/Users/user/Desktop/project/image/train2',
    'val_dir': '/Users/user/Desktop/project/image/vaildation2',  # Update these paths to your actual paths
    'pretrained_weights': 'vit.pth',
    'new_checkpoint_path': 'vit_continued.pth'
}

def set_seed(seed=42):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def load_pretrained_model(model, weights_path, device):
    """Load pretrained weights into model"""
    if not os.path.exists(weights_path):
        print(f"Warning: Weights file {weights_path} not found!")
        return model, 0
        
    try:
        checkpoint = torch.load(weights_path, map_location=device)
        
        if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
            start_epoch = checkpoint.get('epoch', 0) + 1
            print(f"Loaded checkpoint from epoch {start_epoch-1}")
        else:
            model.load_state_dict(checkpoint)
            start_epoch = 0
            print("Loaded model weights")
            
        print(f"Successfully loaded weights from {weights_path}")
        return model, start_epoch
    except Exception as e:
        print(f"Error loading weights: {e}")
        print("Starting with fresh model")
        return model, 0

def plot_training_curves(train_losses, val_losses, train_accuracies, val_accuracies, save_dir='./plots'):
    """Plot training and validation loss/accuracy curves"""
    os.makedirs(save_dir, exist_ok=True)
    
    # Plot Loss Curves
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(save_dir, 'loss_curves.png'))
    plt.close()
    
    # Plot Accuracy Curves
    plt.figure(figsize=(10, 5))
    plt.plot(train_accuracies, label='Training Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.title('Accuracy Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(save_dir, 'accuracy_curves.png'))
    plt.close()

def plot_confusion_matrix(y_true, y_pred, save_dir='./plots'):
    """Plot confusion matrix"""
    os.makedirs(save_dir, exist_ok=True)
    
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(os.path.join(save_dir, 'confusion_matrix.png'))
    plt.close()

def plot_roc_curve(y_true, y_prob, save_dir='./plots'):
    """Plot ROC curve"""
    os.makedirs(save_dir, exist_ok=True)
    
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)
    
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(os.path.join(save_dir, 'roc_curve.png'))
    plt.close()

def plot_performance_metrics(precisions, recalls, f1_scores, save_dir='./plots'):
    """Plot performance metrics over epochs"""
    os.makedirs(save_dir, exist_ok=True)
    
    epochs = range(1, len(precisions) + 1)
    
    plt.figure(figsize=(12, 6))
    plt.plot(epochs, precisions, 'b-', label='Precision')
    plt.plot(epochs, recalls, 'r-', label='Recall')
    plt.plot(epochs, f1_scores, 'g-', label='F1-Score')
    plt.title('Performance Metrics Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(save_dir, 'performance_metrics.png'))
    plt.close()

def train_one_epoch(model, train_loader, optimizer, criterion, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    progress_bar = tqdm(train_loader, desc="Training")
    for batch_idx, (data, targets) in enumerate(progress_bar):
        data, targets = data.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs.logits, targets)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.logits.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        # Update progress bar
        progress_bar.set_postfix({
            'loss': total_loss/(batch_idx+1),
            'acc': 100.*correct/total
        })
    
    return total_loss/len(train_loader), correct/total

def validate_with_predictions(model, val_loader, criterion, device):
    """Validation function that returns predictions and probabilities"""
    model.eval()
    total_loss = 0
    all_predictions = []
    all_targets = []
    all_probabilities = []
    
    with torch.no_grad():
        for data, targets in tqdm(val_loader, desc="Validating"):
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            loss = criterion(outputs.logits, targets)
            
            total_loss += loss.item()
            probabilities = torch.softmax(outputs.logits, dim=1)
            _, predicted = outputs.logits.max(1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            all_probabilities.extend(probabilities[:, 1].cpu().numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(all_targets, all_predictions)
    precision = precision_score(all_targets, all_predictions, average='weighted', zero_division=0)
    recall = recall_score(all_targets, all_predictions, average='weighted', zero_division=0)
    f1 = f1_score(all_targets, all_predictions, average='weighted', zero_division=0)
    
    return (total_loss/len(val_loader), accuracy, precision, recall, f1, 
            all_targets, all_predictions, all_probabilities)

def main():
    # Set random seed and device
    set_seed(CONFIG['seed'])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    try:
        # Initialize model
        print("Initializing model...")
        model = ViTForImageClassification.from_pretrained(
            CONFIG['model_name'],
            num_labels=2,
            ignore_mismatched_sizes=True
        )
        
        # Custom classifier head
        model.classifier = nn.Sequential(
            nn.Linear(model.config.hidden_size, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 2)
        )
        
        # Load pre-trained weights
        print("Loading pre-trained weights...")
        model, start_epoch = load_pretrained_model(model, CONFIG['pretrained_weights'], device)
        model.to(device)
        
        # Data transforms
        train_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        val_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        # Load datasets
        print("Loading datasets...")
        train_dataset = ImageFolder(CONFIG['train_dir'], transform=train_transform)
        val_dataset = ImageFolder(CONFIG['val_dir'], transform=val_transform)
        
        print(f"Number of training samples: {len(train_dataset)}")
        print(f"Number of validation samples: {len(val_dataset)}")
        
        train_loader = DataLoader(
            train_dataset, 
            batch_size=CONFIG['batch_size'], 
            shuffle=True, 
            num_workers=0
        )
        val_loader = DataLoader(
            val_dataset, 
            batch_size=CONFIG['batch_size'], 
            shuffle=False, 
            num_workers=0
        )
        
        # Initialize optimizer and criterion
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=CONFIG['learning_rate'],
            weight_decay=CONFIG['weight_decay']
        )
        criterion = nn.CrossEntropyLoss()
        
        # Training metrics storage
        train_losses = []
        val_losses = []
        train_accuracies = []
        val_accuracies = []
        precisions = []
        recalls = []
        f1_scores = []
        best_val_accuracy = 0
        
        print("Starting training...")
        for epoch in range(start_epoch, CONFIG['num_epochs']):
            print(f"\nEpoch {epoch+1}/{CONFIG['num_epochs']}")
            
            # Train
            train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
            
            # Validate with predictions
            (val_loss, val_acc, precision, recall, f1,
             targets, predictions, probabilities) = validate_with_predictions(
                model, val_loader, criterion, device
            )
            
            # Store metrics
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            train_accuracies.append(train_acc)
            val_accuracies.append(val_acc)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1)
            
            print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%")
            print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%")
            print(f"Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")
            
            # Save best model
            if val_acc > best_val_accuracy:
                best_val_accuracy = val_acc
                print(f"Saving best model with validation accuracy: {val_acc*100:.2f}%")
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'val_accuracy': val_acc,
                }, CONFIG['new_checkpoint_path'])
        
        # Plot all visualizations
        print("\nGenerating visualization plots...")
        plot_training_curves(train_losses, val_losses, train_accuracies, val_accuracies)
        plot_confusion_matrix(targets, predictions)
        plot_roc_curve(targets, probabilities)
        plot_performance_metrics(precisions, recalls, f1_scores)
        print("All plots have been saved in the 'plots' directory")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        import traceback
        print(traceback.format_exc())

if __name__ == "__main__":
    main()

Using device: cuda
Initializing model...


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading pre-trained weights...
Loaded checkpoint from epoch 8
Successfully loaded weights from vit.pth
Loading datasets...
Number of training samples: 1285
Number of validation samples: 300
Starting training...

Epoch 10/100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:50<00:00,  2.63s/it]


Train Loss: 0.0369 | Train Acc: 98.29%
Val Loss: 0.5091 | Val Acc: 85.33%
Precision: 0.8574 | Recall: 0.8533 | F1: 0.8529
Saving best model with validation accuracy: 85.33%

Epoch 11/100


...

Train Loss: 0.0256 | Train Acc: 98.44%
Val Loss: 1.0099 | Val Acc: 86.33%
Precision: 0.8721 | Recall: 0.8633 | F1: 0.8625

Epoch 99/100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:51<00:00,  2.72s/it]


Train Loss: 0.0256 | Train Acc: 98.44%
Val Loss: 0.8380 | Val Acc: 87.67%
Precision: 0.8816 | Recall: 0.8767 | F1: 0.8763

Epoch 100/100


Validating: 100%|██████████████████████████████████████████████████████████████████████| 19/19 [00:50<00:00,  2.65s/it]


Train Loss: 0.0250 | Train Acc: 98.44%
Val Loss: 0.8220 | Val Acc: 87.67%
Precision: 0.8816 | Recall: 0.8767 | F1: 0.8763

Generating visualization plots...
All plots have been saved in the 'plots' directory
